In [3]:
# %%writefile senate_vote_scraper.py
from pymongo import MongoClient
import pprint 
import pandas as pd 
import copy
from bs4 import BeautifulSoup
import pprint
import requests
import datetime
import re

from pyvirtualdisplay import Display
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

from my_tools import write_json_file

In [ ]:
# https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_101_1.htm
# can I employ multithreading to get these quicker?

In [ ]:
def get_session_summaries(cong_id, session):
    root_url = 'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu'

    site_url = '{}_{}_{}.htm'.format(root_url, cong_id, session)
    req = requests.get(site_url)
    tstamp = datetime.datetime.now().strftime('%m-%d-%Y %H:%M:%S')
    stat_code = req.status_code
    if stat_code != 200:
        print('_______________')
        print('_______________')
        print('Error requesting {}'.format(site_url))
        print('Request Status Code: {}, {}'.format(stat_code, tstamp))

    if stat_code == 200:            
        # use BeautifulSoup to find the data we need.
        soup = BeautifulSoup(req.content, 'lxml')
        table = soup.find('table')
        rows = table.find_all('tr')
        
        outfile = '../data/senate_vote_results_{}_{}.jsonl'.format(cong_id, session)
        append_rows_to_file(cong_id, session, rows, outfile)

        
    print('\tIterations through rolls for cong_id {}, session {} complete.'.format(cong_id, session))
    print('\tLast url: {}'.format(site_url))
    print("\tExamine output above for occurrences in request errors, if any.")
    print('_______________')


In [ ]:
def append_rows_to_file(cong_id, session, rows, filename):
    # create dictionaries to get year from cong_id and session
    cong_ids = range(101, 116)
    years_odd = range(1989, 2019, 2)
    years_even = range(1990, 2019, 2)

    s1_congid_dict = {}
    for i, j in zip(cong_ids, years_odd):
        s1_congid_dict.update({i:j})

    s2_congid_dict = {}
    for i, j in zip(cong_ids, years_even):
        s2_congid_dict.update({i:j})
    
    if session%2 == 0:
        yr = s2_congid_dict[cong_id]
    else:
        yr = s1_congid_dict[cong_id]
    
    
    
    # initialize an empty list
#     all_records = []

    
    # create an empty row to append to all_records with info filled in
    empty_row = {
        'congress_id': None,
        'session': None,
        'vote_id': None, 
        'issue': None, 
        'result': None, 
        'question': None, 
        'desc': None, 
        'date': None,
        'year': None, 
        'vote_results': None
    }

    
    for i in range(len(rows)):
        columns = rows[i].find_all('td')
        if len(columns) > 0:
            leg_id = columns[3].text

            if ((leg_id.startswith('S. ')) | 
                (leg_id.startswith('S.J.Res')) | 
                (leg_id.startswith('H.R ')) |  
                (leg_id.startswith('H.J.Res'))):
                leg_id = leg_id.replace('.', ' ').replace('  ', ' ').upper().strip()
                print('---------')
                print(leg_id)
                print('---------')


                new_row = copy.copy(empty_row)
                columns = rows[i].find_all('td')
                new_row['congress_id'] = cong_id
                new_row['session'] = session
                new_row['vote_id'] = re.sub(r'[^\x00-\x7F]+', ' ' ,columns[0].text).strip().split(' ')[0]
                new_row['issue'] = leg_id
                new_row['result'] = columns[1].text
                new_row['question'] = columns[2].text.split(':', 1)[0]
                new_row['desc'] = columns[2].text.split(':', 1)[1][1:]
                new_row['date'] = re.sub(r'[^\x00-\x7F]+', ' ' ,columns[4].text).strip()
                new_row['year'] = yr
    #             new_row['vote_results'] = None

#                 all_records.append(new_row)

                write_json_file(new_row, filename)


In [ ]:
get_session_summaries(101, 1)

In [5]:
# send GET request using selenium (sites in javascript) and check status
option = webdriver.ChromeOptions()
option.add_argument(' - incognito')
option.add_argument('--headless')
option.add_argument('--no-sandbox')


browser = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver', chrome_options=option)


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally
  (Driver info: chromedriver=2.26.436382 (70eb799287ce4c2208441fc057053a5b07ceabac),platform=Linux 4.15.0-1029-aws x86_64)


In [ ]:
site_url = 'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_101_1.htm/'
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
req = requests.get(site_url, headers=headers)
tstamp = datetime.datetime.now().strftime('%m-%d-%Y %H:%M:%S')
stat_code = req.status_code


In [ ]:
stat_code

In [ ]:
import re

In [ ]:
columns[2].text.split(':', 1)[1][1:]

In [ ]:
re.sub(r'[^\x00-\x7F]+', ' ' ,columns[0].text).strip().split(' ')[0]

In [ ]:
columns = rows[1].find_all('a')
for col in columns:
    print('--------')
    print(col)

In [ ]:
columns